<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 3 - 3 Hours </h1>
<h1 style="text-align:center">Long Short Term Memory (LSTM) for Language Modeling</h1>

<b> Student 1:</b> Collura  
<b> Student 2:</b> Spano
 
 
In this Lab Session,  you will build and train a Recurrent Neural Network, based on Long Short-Term Memory (LSTM) units for next word prediction task. 

Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 
Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by June 9th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab3]** as Subject of your email.

#  Introduction

You will train a LSTM to predict the next word using a sample short story. The LSTM will learn to predict the next item of a sentence from the 3 previous items (given as input). Ponctuation marks are considered as dictionnary items so they can be predicted too. Figure 1 shows the LSTM and the process of next word prediction. 

<img src="lstm.png" height="370" width="370"> 


Each word (and ponctuation) from text sentences is encoded by a unique integer. The integer value corresponds to the index of the corresponding word (or punctuation mark) in the dictionnary. The network output is a one-hot-vector indicating the index of the predicted word in the reversed dictionnary (Section 1.2). For example if the prediction is 86, the predicted word will be "company". 



You will use a sample short story from Aesop’s Fables (http://www.taleswithmorals.com/) to train your model. 


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







Start by loading the necessary libraries and resetting the default computational graph. For more details about the rnn packages, we suggest you to take a look at https://www.tensorflow.org/api_guides/python/contrib.rnn

In [1]:
import numpy as np
import collections # used to build the dictionary
import random
import time
import pickle # may be used to save your model 
import matplotlib.pyplot as plt
#Import Tensorflow and rnn
import tensorflow as tf
from tensorflow.contrib import rnn  

# Target log path
logs_path = 'lstm_words'
writer = tf.summary.FileWriter(logs_path)

# Next-word prediction task

## Part 1: Data  preparation

### 1.1. Loading data

Load and split the text of our story

In [2]:
def load_data(filename):
    with open(filename) as f:
        data = f.readlines()
    data = [x.strip().lower() for x in data]
    data = [data[i].split() for i in range(len(data))]
    data = np.array(data)
    data = np.reshape(data, [-1, ])
    print(data)
    return data

#Run the cell 
train_file ='data/story.txt'
train_data = load_data(train_file)
print("Loaded training data...")
print(len(train_data))

['there' 'was' 'once' 'a' 'young' 'shepherd' 'boy' 'who' 'tended' 'his'
 'sheep' 'at' 'the' 'foot' 'of' 'a' 'mountain' 'near' 'a' 'dark' 'forest'
 '.' 'it' 'was' 'rather' 'lonely' 'for' 'him' 'all' 'day' ',' 'so' 'he'
 'thought' 'upon' 'a' 'plan' 'by' 'which' 'he' 'could' 'get' 'a' 'little'
 'company' 'and' 'some' 'excitement' '.' 'he' 'rushed' 'down' 'towards'
 'the' 'village' 'calling' 'out' 'wolf' ',' 'wolf' ',' 'and' 'the'
 'villagers' 'came' 'out' 'to' 'meet' 'him' ',' 'and' 'some' 'of' 'them'
 'stopped' 'with' 'him' 'for' 'a' 'considerable' 'time' '.' 'this'
 'pleased' 'the' 'boy' 'so' 'much' 'that' 'a' 'few' 'days' 'afterwards'
 'he' 'tried' 'the' 'same' 'trick' ',' 'and' 'again' 'the' 'villagers'
 'came' 'to' 'his' 'help' '.' 'but' 'shortly' 'after' 'this' 'a' 'wolf'
 'actually' 'did' 'come' 'out' 'from' 'the' 'forest' ',' 'and' 'began' 'to'
 'worry' 'the' 'sheep,' 'and' 'the' 'boy' 'of' 'course' 'cried' 'out'
 'wolf' ',' 'wolf' ',' 'still' 'louder' 'than' 'before' '.' 'but' 't

### 1.2. Symbols encoding

The LSTM input's can only be numbers. A way to convert words (symbols or any items) to numbers is to assign a unique integer to each word. This process is often based on frequency of occurrence for efficient coding purpose.

Here, we define a function to build an indexed word dictionary (word->number). The "build_vocabulary" function builds both:

- Dictionary : used for encoding words to numbers for the LSTM inputs 
- Reverted dictionnary : used for decoding the outputs of the LSTM into words (and punctuation).

For example, in the story above, we have **113** individual words. The "build_vocabulary" function builds a dictionary with the following entries ['the': 0], [',': 1], ['company': 85],...


In [3]:
def build_vocabulary(words):
    count = collections.Counter(words).most_common()
    dic= dict()
    for word, _ in count:
        dic[word] = len(dic)
    reverse_dic = dict(zip(dic.values(), dic.keys()))
    return dic, reverse_dic

Run the cell below to display the vocabulary

In [6]:
dictionary, reverse_dictionary = build_vocabulary(train_data)
vocabulary_size= len(dictionary) 
print("Dictionary size (Vocabulary size) = ", vocabulary_size)
print("\n")
print("Dictionary : \n")
print(dictionary)
print("\n")
print("Reverted Dictionary : \n" )
print(reverse_dictionary)

Dictionary size (Vocabulary size) =  113


Dictionary : 

{'was': 11, 'that': 101, 'had': 32, 'help': 17, 'afterwards': 33, 'when': 18, 'all': 34, 'again': 25, 'good': 36, 'stirred': 38, 'young': 39, 'forest': 19, 'much': 40, 'some': 27, 'foot': 42, 'sheep,': 43, 'course': 93, 'near': 45, 'shortly': 46, 'meet': 95, 'rather': 47, 'village': 20, ':': 48, 'them': 21, 'for': 22, 'so': 12, 'plan': 49, 'considerable': 50, 'truth': 51, 'there': 52, 'not': 53, 'twice': 54, 'day': 55, 'time': 23, 'meal': 56, 'at': 57, 'fooled': 58, 'began': 72, 'speaks': 59, 'louder': 60, 'excitement': 61, 'after': 62, 'once': 63, ',': 1, 'sheep': 64, 'which': 65, 'rushed': 66, 'deceiving': 67, 'and': 3, 'dark': 69, 'who': 28, 'actually': 73, 'little': 70, 'few': 71, 'shepherd': 35, 'before': 24, 'out': 9, 'made': 37, 'wise': 74, 'mountain': 83, 'nobody': 76, 'of': 10, 'him': 14, 'down': 77, 'upon': 78, 'wolf': 5, 'towards': 79, 'flock': 80, 'get': 82, 'thought': 26, 'liar': 75, 'villagers': 15, 'pleased': 84, 

## Part 2 : LSTM Model in TensorFlow

Since you have defined how the data will be modeled, you are now to develop an LSTM model to predict the word of following a sequence of 3 words. 

### 2.1. Model definition

Define a 2-layers LSTM model.  

For this use the following classes from the tensorflow.contrib library:

- rnn.BasicLSTMCell(number of hidden units) 
- rnn.static_rnn(rnn_cell, data, dtype=tf.float32)
- rnn.MultiRNNCell(,)


You may need some tensorflow functions (https://www.tensorflow.org/api_docs/python/tf/) :
- tf.split
- tf.reshape 
- ...




In [127]:
def lstm_model(x, w, b, n_hidden=265, n_input=3, n_layers=2):
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x, n_input, 1)

    # 1-layer LSTM with n_hidden units.
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden) for _ in range(n_layers)])

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], w['out']) + b['out']

Training Parameters and constants

In [128]:
tf.reset_default_graph()
# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 3
model_path = 'lstm_model/n%d' % n_input

# For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size]))}

# build the model
pred = lstm_model(x, weights, biases, n_hidden=n_hidden, n_input=n_input)

Define the Loss/Cost and optimizer

In [129]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)  # use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

We give you here the Test Function

In [130]:
#run the cell
def test(sentence, session, verbose=False):
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        print("sentence length should be equal to", n_input, "!")
    try:
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
        words.append(reverse_dictionary[onehot_pred_index])
        sentence = " ".join(words)
        if verbose:
            print(sentence)
        return reverse_dictionary[onehot_pred_index]
    except:
        print(" ".join(["Word", words[i - n_input], "not in dictionary"]))

## Part 3 : LSTM Training  

In the Training process, at each epoch, 3 words are taken from the training data, encoded to integer to form the input vector. The training labels are one-hot vector encoding the word that comes after the 3 inputs words. Display the loss and the training accuracy every 1000 iteration. Save the model at the end of training in the **lstm_model** folder

In [131]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

# Launch the graph
with tf.Session() as session:
    session.run(init)

    print("Start Training")
    ##############################################
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    

    writer.add_graph(session.graph)

    while step < epochs:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
 
    ##############################################
    
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    # save your model 
    ##############################################
    model_saver.save(session, model_path)
    print("Model saved")

Start Training
Iter= 1000, Average Loss= 4.395913, Average Accuracy= 7.10%
['village', 'said', ':'] - [a] vs [the]
Iter= 2000, Average Loss= 3.757541, Average Accuracy= 12.90%
[',', 'and', 'when'] - [the] vs [.]
Iter= 3000, Average Loss= 2.944651, Average Accuracy= 24.50%
['flock', ',', 'and'] - [when] vs [when]
Iter= 4000, Average Loss= 2.781739, Average Accuracy= 33.90%
['the', 'village', 'said'] - [:] vs [,]
Iter= 5000, Average Loss= 2.740451, Average Accuracy= 33.10%
['young', 'shepherd', 'boy'] - [who] vs [who]
Iter= 6000, Average Loss= 2.146430, Average Accuracy= 47.20%
['near', 'a', 'dark'] - [forest] vs [the]
Iter= 7000, Average Loss= 1.928356, Average Accuracy= 49.10%
['rather', 'lonely', 'for'] - [him] vs [a]
Iter= 8000, Average Loss= 1.673115, Average Accuracy= 55.50%
['he', 'could', 'get'] - [a] vs [for]
Iter= 9000, Average Loss= 1.572587, Average Accuracy= 58.10%
['for', 'him', 'all'] - [day] vs [the]
Iter= 10000, Average Loss= 1.191246, Average Accuracy= 68.20%
['day', ',

## Part 4 : Test your model 

### 3.1. Next word prediction

Load your model (using the model_saved variable given in the training session) and test the sentences :
- 'get a little' 
- 'nobody tried to'
- Try with other sentences using words from the story's vocabulary. 

In [132]:
with tf.Session() as session:
    model_saver.restore(session, model_path)

    sentences = ['get a little', 'nobody tried to']
    for sentence in sentences:
        test(sentence, session, True)
    
    print('\n')
    sentences = [train_data[n:n + n_input + 1] for n in np.random.randint(0, len(train_data) - n_input - 1, 10)]
    for sentence in sentences:
        predicted = test(' '.join(sentence[:-1]), session, True)
        print(' Predicted word:', predicted, '\n Word from text:', sentence[-1])

get a little days
nobody tried to wolf


. but shortly after
 Predicted word: after 
 Word from text: after
trick , and when
 Predicted word: when 
 Word from text: again
company and some excitement
 Predicted word: excitement 
 Word from text: excitement
been fooled twice before
 Predicted word: before 
 Word from text: before
when he speaks the
 Predicted word: the 
 Word from text: the
a little company not
 Predicted word: not 
 Word from text: and
all day , so
 Predicted word: so 
 Word from text: so
he could get a
 Predicted word: a 
 Word from text: a
to his help .
 Predicted word: . 
 Word from text: .
wolf , wolf ,
 Predicted word: , 
 Word from text: ,


### 3.2. More fun with the Fable Writer !

You will use the RNN/LSTM model learned in the previous question to create a
new story/fable.
For this you will choose 3 words from the dictionary which will start your
story and initialize your network. Using those 3 words the RNN will generate
the next word or the story. Using the last 3 words (the newly predicted one
and the last 2 from the input) you will use the network to predict the 5
word of the story.. and so on until your story is 5 sentence long. 
Make a point at the end of your story. 
To implement that, you will use the test function. 

In [133]:
with tf.Session() as session:
    model_saver.restore(session, model_path)
    n = np.random.randint(0, len(train_data) - n_input)
    words = train_data[n:n + n_input].tolist()
    while words.count('.') < 5 and len(words) < 1000:  # set an upper bound on iterations
        sentence = ' '.join(words[-n_input:])
        words.append(test(sentence, session))

if words[-1] != '.':
    words.append('.')
print(' '.join(words))

the wise man of wolf , wolf , still louder than before . but this time the considerable the of so much that a few days not be before . but this time the considerable the of so much that a few days not be before . but this time the considerable the of so much that a few days not be before . but this time the considerable the of so much that a few days not be before .


<div class='alert alert-warning'>
As we can see, we get stuck in a loop. This is due to the fact that the window of words we consider for the prediction is too small: we only look at the previous 3 words to get the next one, so for example, every time we start from "a few days" we will always predict "not", every time we start from "few days not", we will get "he" and so on. We will see if increasing the window of input words to 5 will solve the problem.
</div>

### 3.3. Play with number of inputs

The number of input in our example is 3, see what happens when you use other number (1 and 5)

#### 3.3.1 Number of inputs = 5

In [134]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 5
model_path = 'lstm_model/n%d' % n_input

# For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size]))}

# build the model
pred = lstm_model(x, weights, biases, n_hidden=n_hidden, n_input=n_input)

Define the Loss/Cost and optimizer

In [135]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)  # use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [136]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

# Launch the graph
with tf.Session() as session:
    session.run(init)

    print("Start Training")
    ##############################################
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    

    writer.add_graph(session.graph)

    while step < epochs:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
 
    ##############################################
    
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    # save your model 
    ##############################################
    model_saver.save(session, model_path)
    print("Model saved")

Start Training
Iter= 1000, Average Loss= 4.478766, Average Accuracy= 5.30%
['the', 'wolf', 'made', 'a', 'good'] - [meal] vs [,]
Iter= 2000, Average Loss= 3.535477, Average Accuracy= 18.50%
['boy', 'of', 'course', 'cried', 'out'] - [wolf] vs [,]
Iter= 3000, Average Loss= 2.841516, Average Accuracy= 31.50%
['time', '.', 'this', 'pleased', 'the'] - [boy] vs [boy]
Iter= 4000, Average Loss= 2.312626, Average Accuracy= 44.10%
[',', 'wolf', ',', 'and', 'the'] - [villagers] vs [and]
Iter= 5000, Average Loss= 1.396652, Average Accuracy= 64.20%
['there', 'was', 'once', 'a', 'young'] - [shepherd] vs [shepherd]
Iter= 6000, Average Loss= 0.719507, Average Accuracy= 81.80%
[',', 'the', 'wise', 'man', 'of'] - [the] vs [the]
Iter= 7000, Average Loss= 0.460446, Average Accuracy= 88.60%
['the', 'boy', 'was', 'again', 'deceiving'] - [them] vs [them]
Iter= 8000, Average Loss= 0.333728, Average Accuracy= 91.00%
['same', 'trick', ',', 'and', 'again'] - [the] vs [the]
Iter= 9000, Average Loss= 0.154757, Aver

In [137]:
with tf.Session() as session:
    model_saver.restore(session, model_path)

    sentences = [train_data[n:n + n_input + 1] for n in np.random.randint(0, len(train_data) - n_input - 1, 10)]
    for sentence in sentences:
        predicted = test(' '.join(sentence[:-1]), session, True)
        print(' Predicted word:', predicted, '\n Word from text:', sentence[-1])

complained , the wise man of
 Predicted word: of 
 Word from text: of
for him all day , so
 Predicted word: so 
 Word from text: so
the boy of course cried out
 Predicted word: out 
 Word from text: out
the boy so much that a
 Predicted word: a 
 Word from text: a
but this time the villagers ,
 Predicted word: , 
 Word from text: ,
thought the boy was again deceiving
 Predicted word: deceiving 
 Word from text: deceiving
but shortly after this a wolf
 Predicted word: wolf 
 Word from text: wolf
tended his sheep at the foot
 Predicted word: foot 
 Word from text: foot
forest . it was rather lonely
 Predicted word: lonely 
 Word from text: lonely
, wolf , still louder than
 Predicted word: than 
 Word from text: than


In [138]:
with tf.Session() as session:
    model_saver.restore(session, model_path)
    n = np.random.randint(0, len(train_data) - n_input)
    words = train_data[n:n + n_input].tolist()
    while words.count('.') < 5 and len(words) < 1000:  # set an upper bound on iterations
        sentence = ' '.join(words[-n_input:])
        words.append(test(sentence, session))

if words[-1] != '.':
    words.append('.')
print(' '.join(words))

it was rather lonely for him all day , so he thought upon a plan by which he could get a little company and some excitement . he rushed down towards the village calling out wolf , wolf , and the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to his help . but shortly after this a wolf actually did come out from the forest , and began to worry the sheep, and the boy of course cried out wolf , wolf , and the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to his help .


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







<div class='alert alert-warning'>
As we can see the text we obtain is exactly the same as the original. We believe this is due to overfitting on the training data: in fact, the window of words we train on is too big, and we end up guessing all the right words.
</div>

#### 3.3.2 Number of inputs = 1

In [139]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 1
model_path = 'lstm_model/n%d' % n_input

# For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size]))}

# build the model
pred = lstm_model(x, weights, biases, n_hidden=n_hidden, n_input=n_input)

Define the Loss/Cost and optimizer

In [140]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)  # use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [141]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

# Launch the graph
with tf.Session() as session:
    session.run(init)

    print("Start Training")
    ##############################################
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    

    writer.add_graph(session.graph)

    while step < epochs:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
 
    ##############################################
    
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    # save your model 
    ##############################################
    model_saver.save(session, model_path)
    print("Model saved")

Start Training
Iter= 1000, Average Loss= 4.631672, Average Accuracy= 4.10%
['much'] - [that] vs [.]
Iter= 2000, Average Loss= 4.270354, Average Accuracy= 6.70%
['the'] - [wolf] vs [and]
Iter= 3000, Average Loss= 4.124525, Average Accuracy= 8.60%
['village'] - [calling] vs [a]
Iter= 4000, Average Loss= 4.044169, Average Accuracy= 10.00%
['than'] - [before] vs [the]
Iter= 5000, Average Loss= 3.958994, Average Accuracy= 11.30%
['of'] - [a] vs [,]
Iter= 6000, Average Loss= 4.038328, Average Accuracy= 11.20%
['villagers'] - [came] vs [.]
Iter= 7000, Average Loss= 3.869531, Average Accuracy= 11.60%
['flock'] - [,] vs [the]
Iter= 8000, Average Loss= 4.091427, Average Accuracy= 11.00%
['out'] - [wolf] vs [a]
Iter= 9000, Average Loss= 3.683072, Average Accuracy= 13.70%
[','] - [still] vs [wolf]
Iter= 10000, Average Loss= 3.862956, Average Accuracy= 12.90%
['his'] - [sheep] vs [.]
Iter= 11000, Average Loss= 3.947990, Average Accuracy= 12.50%
['afterwards'] - [he] vs [out]
Iter= 12000, Average Lo

In [142]:
with tf.Session() as session:
    model_saver.restore(session, model_path)

    sentences = [train_data[n:n + n_input + 1] for n in np.random.randint(0, len(train_data) - n_input - 1, 10)]
    for sentence in sentences:
        predicted = test(' '.join(sentence[:-1]), session, True)
        print(' Predicted word:', predicted, '\n Word from text:', sentence[-1])

wolf who
 Predicted word: who 
 Word from text: actually
and some
 Predicted word: some 
 Word from text: some
with was
 Predicted word: was 
 Word from text: him
his a
 Predicted word: a 
 Word from text: help
help a
 Predicted word: a 
 Word from text: .
calling was
 Predicted word: was 
 Word from text: out
come .
 Predicted word: . 
 Word from text: out
, and
 Predicted word: and 
 Word from text: who
thought a
 Predicted word: a 
 Word from text: the
his a
 Predicted word: a 
 Word from text: help


<div class='alert alert-warning'>
As expected given the low accuracy obtained in the training phase, we do not have good guesses on the next words in this case.
</div>

In [143]:
with tf.Session() as session:
    model_saver.restore(session, model_path)
    n = np.random.randint(0, len(train_data) - n_input)
    words = train_data[n:n + n_input].tolist()
    while words.count('.') < 5 and len(words) < 1000:  # set an upper bound on iterations
        sentence = ' '.join(words[-n_input:])
        words.append(test(sentence, session))

if words[-1] != '.':
    words.append('.')
print(' '.join(words))

boy who . the boy who . the boy who . the boy who . the boy who .


<div class='alert alert-warning'>
As happened above with input=3, here we are stuck in a loop and we keep printing the same sentences over again. This is due to the fact that the window is too small, and it predicts only based on the last word: every time it finds "," for example, it will always predict "and" as the following word. Same with "and" and "the" and so on.
</div>

#### 3.3.3 Number of inputs = 4

In [144]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 4
model_path = 'lstm_model/n%d' % n_input

# For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size]))}

# build the model
pred = lstm_model(x, weights, biases, n_hidden=n_hidden, n_input=n_input)

Define the Loss/Cost and optimizer

In [145]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)  # use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [146]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

# Launch the graph
with tf.Session() as session:
    session.run(init)

    print("Start Training")
    ##############################################
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    

    writer.add_graph(session.graph)

    while step < epochs:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
 
    ##############################################
    
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    # save your model 
    ##############################################
    model_saver.save(session, model_path)
    print("Model saved")

Start Training
Iter= 1000, Average Loss= 4.322022, Average Accuracy= 7.70%
['believed', ',', 'even', 'when'] - [he] vs [man]
Iter= 2000, Average Loss= 3.607812, Average Accuracy= 16.80%
['said', ':', 'a', 'liar'] - [will] vs [will]
Iter= 3000, Average Loss= 3.167056, Average Accuracy= 27.20%
['his', 'help', '.', 'so'] - [the] vs [,]
Iter= 4000, Average Loss= 2.415726, Average Accuracy= 42.60%
['boy', 'was', 'again', 'deceiving'] - [them] vs [them]
Iter= 5000, Average Loss= 2.113297, Average Accuracy= 46.80%
['of', 'course', 'cried', 'out'] - [wolf] vs [wolf]
Iter= 6000, Average Loss= 1.343372, Average Accuracy= 65.40%
['.', 'but', 'shortly', 'after'] - [this] vs [,]
Iter= 7000, Average Loss= 0.839098, Average Accuracy= 78.40%
['villagers', 'came', 'to', 'his'] - [help] vs [stopped]
Iter= 8000, Average Loss= 0.794565, Average Accuracy= 80.10%
['that', 'a', 'few', 'days'] - [afterwards] vs [afterwards]
Iter= 9000, Average Loss= 0.441421, Average Accuracy= 88.80%
['a', 'few', 'days', 'aft

In [147]:
with tf.Session() as session:
    model_saver.restore(session, model_path)

    sentences = [train_data[n:n + n_input + 1] for n in np.random.randint(0, len(train_data) - n_input - 1, 10)]
    for sentence in sentences:
        predicted = test(' '.join(sentence[:-1]), session, True)
        print(' Predicted word:', predicted, '\n Word from text:', sentence[-1])

he could get a little
 Predicted word: little 
 Word from text: little
who tended his sheep at
 Predicted word: at 
 Word from text: at
village said : a liar
 Predicted word: liar 
 Word from text: liar
towards the village calling out
 Predicted word: out 
 Word from text: out
was once a young shepherd
 Predicted word: shepherd 
 Word from text: shepherd
mountain near a dark forest
 Predicted word: forest 
 Word from text: forest
. so the wolf made
 Predicted word: made 
 Word from text: made
wolf , still louder than
 Predicted word: than 
 Word from text: than
off the boy's flock ,
 Predicted word: , 
 Word from text: ,
a little company and some
 Predicted word: some 
 Word from text: some


<div class='alert alert-warning'>
As expected given the low accuracy obtained in the training phase, we do not have good guesses on the next words in this case.
</div>

In [148]:
with tf.Session() as session:
    model_saver.restore(session, model_path)
    n = np.random.randint(0, len(train_data) - n_input)
    words = train_data[n:n + n_input].tolist()
    while words.count('.') < 5 and len(words) < 1000:  # set an upper bound on iterations
        sentence = ' '.join(words[-n_input:])
        words.append(test(sentence, session))

if words[-1] != '.':
    words.append('.')
print(' '.join(words))

shortly after this a wolf actually did come out from the forest , and began to worry the sheep, and the boy of course cried out wolf , wolf , and the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a few days afterwards he tried the same trick , and again the villagers came to to meet him , and some of them stopped with him for a considerable time .


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







<div class='alert alert-warning'>
In this case, we can see how we are in a midway between n_input = 3 and n_input = 5. We do not obtain exactly the same text as the original one, but we do not get stuck in a loop. 
</div>

## 3-Layers

In [150]:
tf.reset_default_graph()
# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 3
n_layers = 3
model_path = 'lstm_model/n%d_%d' % (n_input, n_layers)

# For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size]))}

# build the model
pred = lstm_model(x, weights, biases, n_hidden=n_hidden, n_input=n_input, n_layers=n_layers)

Define the Loss/Cost and optimizer

In [151]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))  # Cross Entropy loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)  # use RMSProp Optimizer

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

We give you here the Test Function

In [152]:
#run the cell
def test(sentence, session, verbose=False):
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        print("sentence length should be equal to", n_input, "!")
    try:
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
        words.append(reverse_dictionary[onehot_pred_index])
        sentence = " ".join(words)
        if verbose:
            print(sentence)
        return reverse_dictionary[onehot_pred_index]
    except:
        print(" ".join(["Word", words[i - n_input], "not in dictionary"]))

## Part 3 : LSTM Training  

In the Training process, at each epoch, 3 words are taken from the training data, encoded to integer to form the input vector. The training labels are one-hot vector encoding the word that comes after the 3 inputs words. Display the loss and the training accuracy every 1000 iteration. Save the model at the end of training in the **lstm_model** folder

In [153]:
# Initializing the variables
start_time = time.time()
init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

# Launch the graph
with tf.Session() as session:
    session.run(init)

    print("Start Training")
    ##############################################
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    

    writer.add_graph(session.graph)

    while step < epochs:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(train_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(train_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocabulary_size], dtype=float)
        symbols_out_onehot[dictionary[str(train_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [train_data[i] for i in range(offset, offset + n_input)]
            symbols_out = train_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
 
    ##############################################
    
    print("End Of training Finished!")
    print("time: ",time.time() - start_time)
    print("For tensorboard visualisation run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("and oint your web browser to the returned link")
    ##############################################
    # save your model 
    ##############################################
    model_saver.save(session, model_path)
    print("Model saved")

Start Training
Iter= 1000, Average Loss= 4.513784, Average Accuracy= 7.80%
['not', 'be', 'believed'] - [,] vs [a]
Iter= 2000, Average Loss= 3.925602, Average Accuracy= 9.10%
['he', 'speaks', 'the'] - [truth] vs [boy]
Iter= 3000, Average Loss= 3.067097, Average Accuracy= 25.90%
['who', 'tended', 'his'] - [sheep] vs [.]
Iter= 4000, Average Loss= 2.640464, Average Accuracy= 36.50%
['the', 'foot', 'of'] - [a] vs [,]
Iter= 5000, Average Loss= 2.246926, Average Accuracy= 45.00%
['forest', '.', 'it'] - [was] vs [was]
Iter= 6000, Average Loss= 2.146455, Average Accuracy= 43.70%
['near', 'a', 'dark'] - [forest] vs [forest]
Iter= 7000, Average Loss= 1.503713, Average Accuracy= 59.90%
['near', 'a', 'dark'] - [forest] vs [forest]
Iter= 8000, Average Loss= 1.542260, Average Accuracy= 58.50%
['could', 'get', 'a'] - [little] vs [even]
Iter= 9000, Average Loss= 1.001263, Average Accuracy= 72.80%
['could', 'get', 'a'] - [little] vs [even]
Iter= 10000, Average Loss= 1.069173, Average Accuracy= 70.20%
['

## Part 4 : Test your model 

### 3.1. Next word prediction

Load your model (using the model_saved variable given in the training session) and test the sentences :
- 'get a little' 
- 'nobody tried to'
- Try with other sentences using words from the story's vocabulary. 

In [125]:
with tf.Session() as session:
    model_saver.restore(session, model_path)

    sentences = ['get a little', 'nobody tried to']
    for sentence in sentences:
        test(sentence, session, True)
    
    print('\n')
    sentences = [train_data[n:n + n_input + 1] for n in np.random.randint(0, len(train_data) - n_input - 1, 10)]
    for sentence in sentences:
        predicted = test(' '.join(sentence[:-1]), session, True)
        print(' Predicted word:', predicted, '\n Word from text:', sentence[-1])

get a little company
nobody tried to wolf


, and some the
 Predicted word: the 
 Word from text: of
but this time the
 Predicted word: the 
 Word from text: the
this time the villagers
 Predicted word: villagers 
 Word from text: villagers
had been fooled twice
 Predicted word: twice 
 Word from text: twice
a liar will not
 Predicted word: not 
 Word from text: not
this a wolf actually
 Predicted word: actually 
 Word from text: actually
to his help .
 Predicted word: . 
 Word from text: .
boy so much that
 Predicted word: that 
 Word from text: that
young shepherd boy who
 Predicted word: who 
 Word from text: who
of the village said
 Predicted word: said 
 Word from text: said


### 3.2. More fun with the Fable Writer !

You will use the RNN/LSTM model learned in the previous question to create a
new story/fable.
For this you will choose 3 words from the dictionary which will start your
story and initialize your network. Using those 3 words the RNN will generate
the next word or the story. Using the last 3 words (the newly predicted one
and the last 2 from the input) you will use the network to predict the 5
word of the story.. and so on until your story is 5 sentence long. 
Make a point at the end of your story. 
To implement that, you will use the test function. 

In [126]:
with tf.Session() as session:
    model_saver.restore(session, model_path)
    n = np.random.randint(0, len(train_data) - n_input)
    words = train_data[n:n + n_input].tolist()
    while words.count('.') < 5 and len(words) < 1000:  # set an upper bound on iterations
        sentence = ' '.join(words[-n_input:])
        words.append(test(sentence, session))

if words[-1] != '.':
    words.append('.')
print(' '.join(words))

, the wise man of the village said out wolf , wolf , still louder than before . but this time the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet him , and some the villagers came out to meet h